## 1. Setup

In [ ]:
import os
import glob
import re
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import cv2
from sklearn.metrics import roc_auc_score
from sklearn import model_selection
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import StepLR

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# local notebook settings
# from efficientnet_pytorch_3d import EfficientNet3D
# train_path = 'train_labels.csv'
# test_path = 'sample_submission_csv'
# png_path = 'archive'
# from volumentations import *


# kaggle settings
import sys
sys.path.append('../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D')
sys.path.append('../input/3d-augmentation/volumentations-master')
sys.path.append('../input/gradualwarmupschedulerv2')
sys.path.append('../input/medicalnet')

import sys
from volumentations import *
from efficientnet_pytorch_3d import EfficientNet3D
from warmup_scheduler import GradualWarmupScheduler
from models import resnet
from model import generate_model

train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv'
test_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv'
png_path = '../input/rsna-miccai-png'

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(38)

In [ ]:
img_size = 256
num_img = 64
mri_types = ["FLAIR", "T1w", "T1wCE", "T2w"]
# # mri_types = ["FLAIR", "T1w"]

In [ ]:
# adapt from the original file 
import argparse

def parse_opts():
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--data_root',
        default='./data',
        type=str,
        help='Root directory path of data')
    parser.add_argument(
        '--img_list',
        default='./data/train.txt',
        type=str,
        help='Path for image list file')
    parser.add_argument(
        '--n_seg_classes',
        default=2,
        type=int,
        help="Number of segmentation classes"
    )
    parser.add_argument(
        '--learning_rate',  # set to 0.001 when finetune
        default=0.001,
        type=float,
        help=
        'Initial learning rate (divided by 10 while training by lr scheduler)')
    parser.add_argument(
        '--num_workers',
        default=4,
        type=int,
        help='Number of jobs')
    parser.add_argument(
        '--batch_size', default=1, type=int, help='Batch Size')
    parser.add_argument(
        '--phase', default='train', type=str, help='Phase of train or test')
    parser.add_argument(
        '--save_intervals',
        default=10,
        type=int,
        help='Interation for saving model')
    parser.add_argument(
        '--n_epochs',
        default=200,
        type=int,
        help='Number of total epochs to run')
    parser.add_argument(
        '--input_D',
    default=64,
        type=int,
        help='Input size of depth')
    parser.add_argument(
        '--input_H',
        default=256,
        type=int,
        help='Input size of height')
    parser.add_argument(
        '--input_W',
        default=256,
        type=int,
        help='Input size of width')
    parser.add_argument(
        '--resume_path',
        default='',
        type=str,
        help=
        'Path for resume model.'
    )
    parser.add_argument(
        '--pretrain_path',
        default='pretrain/resnet_50.pth',
        type=str,
        help=
        'Path for pretrained model.'
    )
    parser.add_argument(
        '--new_layer_names',
        #default=['upsample1', 'cmp_layer3', 'upsample2', 'cmp_layer2', 'upsample3', 'cmp_layer1', 'upsample4', 'cmp_conv1', 'conv_seg'],
        default=['conv_seg'],
        type=list,
        help='New layer except for backbone')
    parser.add_argument(
        '--no_cuda', action='store_true', help='If true, cuda is not used.')
    parser.set_defaults(no_cuda=False)
    parser.add_argument(
        '--gpu_id',
        nargs='+',
        type=int,              
        help='Gpu id lists')
    parser.add_argument(
        '--model',
        default='resnet',
        type=str,
        help='(resnet | preresnet | wideresnet | resnext | densenet | ')
    parser.add_argument(
        '--model_depth',
        default=50,
        type=int,
        help='Depth of resnet (10 | 18 | 34 | 50 | 101)')
    parser.add_argument(
        '--resnet_shortcut',
        default='B',
        type=str,
        help='Shortcut type of resnet (A | B)')
    parser.add_argument(
        '--manual_seed', default=38, type=int, help='Manually set random seed')
    parser.add_argument(
        '--ci_test', action='store_true', help='If true, ci testing is used.')
    args = parser.parse_args([])
    args.save_folder = "./trails/models/{}_{}".format(args.model, args.model_depth)
    
    return args

In [ ]:
opts = parse_opts()   
opts.n_epochs = 15
opts.no_cuda = False if torch.cuda.is_available() else True
# opts.pretrain_path = '../input/medicalnet/pretrain/resnet_50_23dataset.pth'
opts.pretrain_path = ''
opts.model_depth = 10
opts.resnet_shortcut = 'B'
opts.input_D = 64
opts.input_H = 256
opts.input_W = 256
opts.gpu_id = [1] if torch.cuda.is_available() else []
opts.batch_size = 4
opts.n_seg_classes = 1

device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')

## 2. Dataloader

In [ ]:
def load_image(scan_id, mri_type, split='train', path=png_path):
    mri = sorted(glob.glob(f"{path}/{split}/{scan_id}/{mri_type}/*.png"), 
                 key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    
    if len(mri) == 0:
        i_img = np.zeros((img_size, img_size, num_img))
    elif len(mri) < num_img:
        i_img = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size))
                        for path in mri[0:num_img]]).T 
        num_zero = num_img - i_img.shape[-1]
        i_img.shape[-1]
        i_img = np.concatenate((i_img, np.zeros((img_size, img_size, num_zero))), -1)
    else: 
        i_img = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size)) 
                         for path in mri[len(mri) // 2 - num_img // 2: len(mri) // 2 + num_img // 2]]).T
            
    return i_img

In [ ]:
def npy_loader(brat, mri_type, mode):
    if mode == "train":
        path = f"../input/rsna-processed-voxels-64x256x256-clahe/voxels/{mri_type}/{brat}.npy"
    else: 
        path = f"../input/rsnaprocessedvoxels64x256x256clahetrain/voxels/{mri_type}/{brat}.npy"
    
    return np.load(path)

In [ ]:
class RSNADataset(Dataset):
    def __init__(self, csv, mri_type, mode='train', transform=None):
        self.csv = csv
        self.mode = mode
        self.transform = transform
        self.mri_type = mri_type
        
    def __len__(self): 
        return self.csv.shape[0]
    
    def __getitem__(self, idx): 
        data = self.csv.iloc[idx]
        brat = str(int(data["BraTS21ID"])).zfill(5)
        mgmt = data['MGMT_value']
        
        image = npy_loader(brat, self.mri_type, self.mode)
#         image = torch.tensor(image, dtype=torch.float32)
#         image = load_image(brat, self.mri_type, split='train')

        image = image.transpose(1, 2, 0)           
    
        if self.transform is not None:
            data = {'image': image}
            aug_data = self.transform(**data)
            image = aug_data['image']
            
        image = image.transpose(2, 0, 1)           
        image = image / 255
        
        if self.mode == 'train': 
            return torch.tensor(image, dtype=torch.float32), torch.tensor(mgmt, dtype=torch.float)
        else: 
            return torch.tensor(image, dtype=torch.float32), brat
                

In [ ]:
transforms_train = Compose([
        ElasticTransformPseudo2D(alpha=360, sigma=360 * 0.05, alpha_affine=360 * 0.03, p=0.3),
        RotatePseudo2D(axes=(0,1), limit=(-30, 30), p=0.5),
        Flip(1, p=0.4)
    ], p=1)

In [ ]:
# # for testing the cell below

# train_df = pd.read_csv(train_path)
# df_train, df_val = model_selection.train_test_split(train_df, test_size=0.2, random_state=38, stratify=train_df["MGMT_value"])
# df_train = df_train.loc[(df_train['BraTS21ID'] != 109) & (df_train['BraTS21ID'] != 123) & (df_train['BraTS21ID'] != 709), :]
# train_dataset = RSNADataset(df_train, mri_types[0], transform=transforms_train)

In [ ]:
# # creates animation to check the dataset

# from matplotlib import animation, rc
# rc('animation', html='jshtml')


# def create_animation(ims):
#     fig = plt.figure(figsize=(3, 3))
#     plt.axis('off')
#     im = plt.imshow(ims[0], cmap="gray")

#     def animate_func(i):
#         im.set_array(ims[i])
#         return [im]

#     return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)

# test = train_dataset.__getitem__(1)
# create_animation(test[0])

In [ ]:
# create_animation(test[0])

## 3. Model

In [ ]:
# EfficientNet3D 
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
        
    def forward(self, x):
        return self.net(x)

In [ ]:
# MedicalNet
class Model(nn.Module):
    def __init__(self, model_name, opts):
        super().__init__()
        self.model_name = model_name
        self.medical_net, parameters = generate_model(opts)
#         self.drop_in = nn.Dropout(p=0.1)
        self.pool = nn.AdaptiveAvgPool3d(1)
        
        if opts.pretrain_path:
            self.init_model()
            
    def forward(self, x):
#         x = self.medical_net(self.drop_in(x))
        x = self.medical_net(x)
        x = self.pool(x)        
        return x
    
    def init_model(self):
        net_dict = self.medical_net.state_dict()
        pretrain = torch.load(f'../input/medicalnet/pretrain/{self.model_name}.pth', 
                              map_location=device)
        pretrain_dict = {k: v for k, v in pretrain['state_dict'].items() if k in net_dict.keys()}
        net_dict.update(pretrain_dict)
        self.medical_net.load_state_dict(net_dict)
        print("loaded pretrained weights")

In [ ]:
model = Model('resnet_50_23dataset', opts)
count = 0
col = [30, 31, 32, 69, 70, 71, 126, 127, 128, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166]

for name, param in model.named_parameters():
#     if param.requires_grad and (count < 160):
    if param.requires_grad and (count not in col):
        param.requires_grad = False
    print(name, param.requires_grad)
    count += 1

model = Model('resnet_10_23dataset', opts)
count = 0

for name, param in model.named_parameters():
    if param.requires_grad and (count < 36):
        param.requires_grad = False
    print(name, param.requires_grad)
    count += 1

## 4. Train 

In [ ]:
class Trainer:
    def __init__(self, mri_type, model, criterion, optimizer, scheduler, device):
        self.mri_type = mri_type
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.model_train_losses = []
        self.model_val_losses = []
        self.model_val_auc = []
        
    def fit(self, train_loader, val_loader, n_epochs):
        best_val = 10

        for epoch in range(n_epochs): 
            self.scheduler.step() 
            
            # train
            avg_train = self.train_epoch(train_loader)
            self.model_train_losses.append(avg_train)
            
            print(f'epoch {epoch + 1} train: {avg_train:.3f}')
                
            # val
            avg_val, auc = self.val_epoch(val_loader)
            
            self.model_val_losses.append(avg_val)
            self.model_val_auc.append(auc)
            print(f'epoch {epoch + 1} val: {avg_val:.3f}, auc: {auc:.3f}')

            if avg_val < best_val: 
                print('save model ...')
                best_val = avg_val
                last_model =  f'{self.mri_type}-e{epoch + 1}-loss{avg_val:.3f}-auc{auc:.3f}.pt'
                torch.save(self.model.state_dict(), f'{self.mri_type}-e{epoch + 1}-loss{avg_val:.3f}-auc{auc:.3f}.pt')

            print('\n\n')
            
        return last_model
                    
    def train_epoch(self, train_loader):
        train_loss = []
        self.model.train()

        for i, data in tqdm(enumerate(train_loader, 0)): 
            x, y = data
            x = torch.unsqueeze(x, dim=1)
            x, y = x.to(device), y.to(device)
            self.optimizer.zero_grad()

            outputs = self.model(x).squeeze(1).view(-1)
            loss = self.criterion(outputs, y)
            loss.backward()
            self.optimizer.step()

            train_loss.append(loss.detach().item())
        avg_train = sum(train_loss) / len(train_loss)
        
        return avg_train

        
    def val_epoch(self, val_loader):
        self.model.eval()
        outputs_all = []
        y_all = []
        val_loss = []    

        for i, data in tqdm(enumerate(val_loader, 0)): 
            with torch.no_grad():
                x, y = data 
                x = torch.unsqueeze(x, dim=1)
                x, y = x.to(device), y.to(device)

                outputs = self.model(x).squeeze(1).view(-1)
                loss = self.criterion(outputs, y)
                
                outputs = torch.sigmoid(outputs)
                outputs_all.extend(outputs.tolist())
                y_all.extend(y.tolist())
                val_loss.append(loss.detach().item())      
        avg_val = sum(val_loss) / len(val_loss)
        auc = roc_auc_score(y_all, outputs_all)
        
        return avg_val, auc

In [ ]:
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

In [ ]:
train_df = pd.read_csv(train_path)
df_train, df_val = model_selection.train_test_split(train_df, test_size=0.2, random_state=38, stratify=train_df["MGMT_value"])
df_train = df_train.loc[(df_train['BraTS21ID'] != 109) & (df_train['BraTS21ID'] != 123) & (df_train['BraTS21ID'] != 709), :]

In [ ]:
cosine_epo = 12
warmup_epo = 3
n_epochs = cosine_epo + warmup_epo

models = []
models_fields = []

for mri_type in mri_types: 
    model = Model('resnet_10_23dataset', opts)
    count = 0

#     for name, param in model.named_parameters():
#         if param.requires_grad and (count < 36):
#             param.requires_grad = False
#         count += 1
        
    criterion = functional.binary_cross_entropy_with_logits
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=8, gamma=0.1)

#     optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

#     scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, cosine_epo)
#     scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

    device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    train_dataset = RSNADataset(df_train, mri_type, transform=transforms_train)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=4, num_workers=4)

    val_dataset = RSNADataset(df_val, mri_type)
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size=4, num_workers=4)

#     mri_trainer = Trainer(mri_type, model, criterion, optimizer, scheduler_warmup, device)
    scheduler_warmup=None
    mri_trainer = Trainer(mri_type, model, criterion, optimizer, scheduler, device)

    last_best_model = mri_trainer.fit(train_loader, val_loader, n_epochs)
    
    models.append(last_best_model)
    models_fields.append(model)

## 5. Predict

In [ ]:
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')

df_test = pd.read_csv(test_path)

write = 0
models = ["../input/best-loss-version5/FLAIR-e14-loss0.666-auc0.635.pt",
         "../input/best-loss-version5/T1w-e13-loss0.671-auc0.650.pt",
         "../input/best-loss-version5/T1wCE-e15-loss0.679-auc0.589.pt",
         "../input/best-loss-version5/T2w-e15-loss0.664-auc0.654.pt"]

mri_types = ["FLAIR", "T1w", "T1wCE", "T2w"]

for trained_model in range(len(models)): 
    model = Model('resnet_10_23dataset', opts)
    model.to(device)
    checkpoint = torch.load(models[trained_model])
    model.load_state_dict(checkpoint)
    model.eval()
    
    test_dataset = RSNADataset(df_test, mri_type=mri_types[trained_model], mode="test")
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4, pin_memory=True)
    
    y_pred = []

    for e, batch in enumerate(test_loader):
        print(f"{e + 1}/{len(test_loader)}", end="\r")
        
        batch, brat = batch
        with torch.no_grad():
            tmp_pred = torch.sigmoid(model(batch.to(device).unsqueeze(1)).view(-1)).squeeze()
            y_pred.extend(tmp_pred.tolist())
    
    y_pred = np.array(y_pred)
    
    if write == 0:
        write = 1
        submission = pd.DataFrame({"BraTS21ID": df_test['BraTS21ID'].apply(lambda x: str(x).zfill(5)), "MGMT_value": y_pred})
    else: 
        submission['MGMT_value'] += y_pred

submission['MGMT_value'] /= len(models)

In [ ]:
submission.to_csv("submission.csv", index=False)